In [1]:
import os

In [2]:
%pwd

'd:\\iNeuron_Projects\\End_to_End_ML_Dental_Implant_Sandblasting\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\iNeuron_Projects\\End_to_End_ML_Dental_Implant_Sandblasting'

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from Dental_Implant_Sandblasting import logger
from Dental_Implant_Sandblasting.utils.common import read_yaml, create_directories
from Dental_Implant_Sandblasting.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    transformed_train_dir: Path
    transformed_test_dir: Path
    test_size: float
    random_state: int
    polynomial_features_degree: int
    scaling_method: str

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            transformed_train_dir=Path(config.root_dir) / "train.csv",
            transformed_test_dir=Path(config.root_dir) / "test.csv",
            test_size=params['test_size'],
            random_state=params['random_state'],
            polynomial_features_degree=params['polynomial_features_degree'],
            scaling_method=params['scaling_method']
        )
        return data_transformation_config

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def load_data(self):
        data = pd.read_csv(self.config.data_path)
        logger.info(f"Data loaded from {self.config.data_path}")
        return data

    def preprocess_data(self, data):
        # Convert columns to numeric, forcing any errors to NaN
        for col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')
        
        # Handle missing values by imputing
        imputer = SimpleImputer(strategy='mean')
        data_imputed = pd.DataFrame(imputer.fit_transform(data))
        data_imputed.columns = data.columns
        
        logger.info("Missing values handled")
        return data_imputed

    def feature_engineering(self, data):
        # Generating polynomial features
        poly = PolynomialFeatures(degree=self.config.polynomial_features_degree, include_bias=False)
        poly_features = poly.fit_transform(data.drop(columns=['(Sa) Average of Surface roughness (micrometer)', 'Cell Viability (%)', 'Result (1=Passed, 0=Failed)']))
        
        # Scaling features
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(poly_features)
        
        # Create a DataFrame with the new features
        feature_columns = poly.get_feature_names_out(data.columns[:-3])
        engineered_data = pd.DataFrame(scaled_features, columns=feature_columns)
        
        # Add target columns back to the DataFrame
        engineered_data['(Sa) Average of Surface roughness (micrometer)'] = data['(Sa) Average of Surface roughness (micrometer)'].values
        engineered_data['Cell Viability (%)'] = data['Cell Viability (%)'].values
        engineered_data['Result (1=Passed, 0=Failed)'] = data['Result (1=Passed, 0=Failed)'].values
        
        logger.info("Feature engineering completed")
        return engineered_data

    def train_test_splitting(self, data):
        train, test = train_test_split(data, test_size=self.config.test_size, random_state=self.config.random_state)

        train.to_csv(self.config.transformed_train_dir, index=False)
        test.to_csv(self.config.transformed_test_dir, index=False)

        logger.info(f"Train-test split completed with train shape: {train.shape} and test shape: {test.shape}")

    def execute(self):
        data = self.load_data()
        preprocessed_data = self.preprocess_data(data)
        engineered_data = self.feature_engineering(preprocessed_data)
        self.train_test_splitting(engineered_data)

# Pipeline execution
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.execute()
except Exception as e:
    logger.exception(e)
    raise e


[2024-07-06 18:20:49,409: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-06 18:20:49,453: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-06 18:20:49,458: INFO: common: created directory at: artifacts]
[2024-07-06 18:20:49,461: INFO: common: created directory at: artifacts/data_transformation]
[2024-07-06 18:20:49,474: INFO: 3810269099: Data loaded from artifacts\data_ingestion\Sandblasting-Condition.csv]
[2024-07-06 18:20:49,525: INFO: 3810269099: Missing values handled]
[2024-07-06 18:20:49,544: INFO: 3810269099: Feature engineering completed]
[2024-07-06 18:20:49,589: INFO: 3810269099: Train-test split completed with train shape: (111, 30) and test shape: (28, 30)]
